In [ ]:
#To cleanup:
#average/median play time, forever and two weeks
#metacritic score-Sam
#user score
#price-Greg
#tags
#positive 
#if you play longer on average, do you have more positive reviews?

In [7]:
%matplotlib inline
from pathlib import Path
import pandas as pd
import scipy.stats as stats

In [8]:
game_df = pd.read_csv("games.csv")
game_df.head()

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,Galactic Bowling is an exaggerated and stylize...,['English'],...,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
1,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
2,1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,0,4.99,0,Jolt Project: The army now has a new robotics ...,"['English', 'Portuguese - Brazil']",...,0,0,0,Campião Games,Campião Games,Single-player,"Action,Adventure,Indie,Strategy",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
3,1355720,Henosis™,"Jul 23, 2020",0 - 20000,0,0,5.99,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Odd Critter Games,Odd Critter Games,"Single-player,Full controller support","Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
4,1139950,Two Weeks in Painland,"Feb 3, 2020",0 - 20000,0,0,0.00,0,ABOUT THE GAME Play as a hacker who has arrang...,"['English', 'Spanish - Spain']",...,0,0,0,Unusual Games,Unusual Games,"Single-player,Steam Achievements","Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


In [10]:
game_df.columns

Index(['AppID', 'Name', 'Release date', 'Estimated owners', 'Peak CCU',
       'Required age', 'Price', 'DLC count', 'About the game',
       'Supported languages', 'Full audio languages', 'Reviews',
       'Header image', 'Website', 'Support url', 'Support email', 'Windows',
       'Mac', 'Linux', 'Metacritic score', 'Metacritic url', 'User score',
       'Positive', 'Negative', 'Score rank', 'Achievements', 'Recommendations',
       'Notes', 'Average playtime forever', 'Average playtime two weeks',
       'Median playtime forever', 'Median playtime two weeks', 'Developers',
       'Publishers', 'Categories', 'Genres', 'Tags', 'Screenshots', 'Movies'],
      dtype='object')

In [24]:
top_scores_df = game_df.sort_values(by="Positive", ascending=False)
top_scores_df = top_scores_df[(top_scores_df["Metacritic score"])>0]
top_scores_df[["Name","Metacritic score","Positive","Negative"]]

,Name,Metacritic score,Positive,Negative
46158,Counter-Strike: Global Offensive,83,5764420,766677
17585,Dota 2,90,1477153,300437
57205,Grand Theft Auto V,96,1171197,210154
46416,Terraria,83,964983,21044
30583,Team Fortress 2,92,823693,56683
...,...,...,...,...
35669,Fester Mudd: Curse of the Gold - Episode 1,76,0,0
33297,Metro Exodus,85,0,0
10807,ElecHead,84,0,0
46236,Guild Wars Factions®,84,0,0


In [26]:
genre_scores_df = top_scores_df.groupby("Genres")["Metacritic score"].mean()
genre_scores_df.head()

Genres
Action                                                                           73.494186
Action,Adventure                                                                 74.662222
Action,Adventure,Casual                                                          75.285714
Action,Adventure,Casual,Free to Play,Indie,Massively Multiplayer,RPG             64.000000
Action,Adventure,Casual,Free to Play,Indie,Massively Multiplayer,RPG,Strategy    76.000000
Name: Metacritic score, dtype: float64

In [44]:
steam_tags_onehot_df = top_scores_df['Genres'].str.split(",").str.join('|').str.get_dummies()
big_genre_df = steam_tags_onehot_df.join(top_scores_df)

action_df = big_genre_df.groupby(big_genre_df["Action"])["Metacritic score"].mean()
adventure_df = big_genre_df.groupby(big_genre_df["Adventure"])["Metacritic score"].mean()
casual_df = big_genre_df.groupby(big_genre_df["Casual"])["Metacritic score"].mean()
eacess_df = big_genre_df.groupby(big_genre_df["Early Access"])["Metacritic score"].mean()
free_df = big_genre_df.groupby(big_genre_df["Free to Play"])["Metacritic score"].mean()
indie_df = big_genre_df.groupby(big_genre_df["Indie"])["Metacritic score"].mean()
multi_df = big_genre_df.groupby(big_genre_df["Massively Multiplayer"])["Metacritic score"].mean()


print(action_df)
print(adventure_df)
print(casual_df)
print(eacess_df)
print(free_df)
print(indie_df)
print(multi_df)

Action
0    72.987186
1    72.591004
Name: Metacritic score, dtype: float64
Adventure
0    72.928135
1    72.605676
Name: Metacritic score, dtype: float64
Casual
0    72.845621
1    72.365801
Name: Metacritic score, dtype: float64
Early Access
0    72.788777
1    71.857143
Name: Metacritic score, dtype: float64
Free to Play
0    72.756296
1    74.021739
Name: Metacritic score, dtype: float64
Indie
0    73.619965
1    72.098456
Name: Metacritic score, dtype: float64
Massively Multiplayer
0    72.775202
1    73.373333
Name: Metacritic score, dtype: float64


In [45]:
tag_scores_df = top_scores_df.groupby("Tags")["Metacritic score"].mean()
tag_scores_df.head()

Tags
1990's,America,Platformer,Action,Relaxing,Exploration,Cute,Puzzle-Platformer,Physics,Family Friendly,Drama,Time Travel,Indie,Controller,Multiple Endings,Atmospheric,Colorful,Choices Matter,Racing,Singleplayer     79.0
1990's,Comedy,Crime,Indie,Singleplayer,Funny,Cartoony,Stylized,Story Rich,Hand-drawn,Female Protagonist,Adventure,Colorful,Choices Matter,Conversation,2D,Cute,Exploration,Action-Adventure,Interactive Fiction      77.0
1990's,Old School,2D,Cartoony,Family Friendly,Casual,Precision Platformer,Difficult,Pixel Graphics,Retro,Indie,Platformer,2D Platformer,Fast-Paced,Singleplayer,Adventure,Action,Female Protagonist,Colorful,Cute    82.0
1990's,Point & Click,Cult Classic,Adventure,Retro,Classic,Remake,Pirates,Singleplayer,Cartoony,Comedy,2D,Colorful,Puzzle,Atmospheric,Great Soundtrack,Funny                                                          89.0
1990's,Side Scroller,Beat 'em up,Martial Arts,2.5D,Historical,Swordplay,Action,World War I,Rome,Western,Pirates,Ninja,Sci-f

In [46]:
steam_actual_tags_onehot_df = top_scores_df['Tags'].str.split(",").str.join('|').str.get_dummies()
steam_actual_tags_onehot_df

,1980s,1990's,2.5D,2D,2D Fighter,2D Platformer,3D,3D Fighter,3D Platformer,3D Vision,...,Well-Written,Werewolves,Western,Wholesome,Word Game,World War I,World War II,Wrestling,Zombies,e-sports
46158,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
17585,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
57205,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
46416,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30583,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35669,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33297,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10807,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
46236,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
